In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.callbacks import CSVLogger
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, utils

model = Sequential()

model.add(Conv2D(16, (10, 10), padding='same', use_bias=False, input_shape=(256, 256, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(32, (5, 5), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (4, 4), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dropout(0.2))
 
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
 
 
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator
 
# train_datagen = ImageDataGenerator(rescale = 1./255)
 
# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.7,
                                   zoom_range=[0.9, 2.2],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.33)
 
training_set = train_datagen.flow_from_directory('./data_resize',
                                                 shuffle=True,
                                                 seed=13,
                                                 target_size = (256, 256),
                                                 batch_size = 15,
                                                 class_mode = 'categorical',
                                                 subset="training")
validation_set = train_datagen.flow_from_directory('./data_resize',
                                                 shuffle=True,
                                                 seed=13,
                                                 target_size = (256, 256),
                                                 batch_size = 10,
                                                 class_mode = 'categorical',
                                                 subset="validation")
 
csv_logger = CSVLogger('./log.csv', append=True, separator=';')

hist = model.fit_generator(training_set,
                         steps_per_epoch = 20,
                         epochs = 5,
                         validation_data = validation_set,
                         validation_steps = 10,
                         callbacks=[csv_logger])
from keras.models import load_model
 
model.save('cnn_attraction_keras_model.h5')
 
# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)
 
# 모델 평가하기
print("-- Evaluate --")
 
scores = model.evaluate_generator(
            validation_set,
            steps = 10)
 
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
 
# 모델 예측하기
print("-- Predict --")
 
output = model.predict_generator(
            validation_set,
            steps = 10)
print(validation_set.class_indices)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
 
print(output)
print(validation_set.filenames)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 256, 256, 16)      4800      
_________________________________________________________________
batch_normalization_9 (Batch (None, 256, 256, 16)      48        
_________________________________________________________________
activation_9 (Activation)    (None, 256, 256, 16)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 128, 128, 32)      12800     
_________________________________________________________________
batch_normalization_10 (Batc (None, 128, 128, 32)     

KeyboardInterrupt: 